In [1]:
import pandas
import string
import nltk
import keras
import sklearn
import time
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
import numpy as np
from time import time
from nltk.corpus import stopwords
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
import keras.preprocessing.text

In [2]:
Train_X = pandas.read_csv("../Dataset/cleaned_train.csv", header=0)['text']
Test_X = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['text']

Train_Y_stance= pandas.read_csv("../Dataset/cleaned_train.csv", header=0)['stance']
Train_Y_category= pandas.read_csv("../Dataset/cleaned_train.csv", header=0)['category']

Test_Y_stance= pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['stance']
Test_Y_category= pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['category']

In [3]:
# concatenate both dev and train texts
texts = Train_X.append(Test_X, ignore_index=True)


tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(texts)

Train_X = tokenizer.texts_to_matrix(Train_X, mode='tfidf')
Test_X = tokenizer.texts_to_matrix(Test_X, mode='tfidf')

C:\Users\hazem\AppData\Local\Temp\ipykernel_13452\1537059493.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts = Train_X.append(Test_X, ignore_index=True)


In [4]:
# concatenate the stance labels in dev and train
tags = Train_Y_stance.append(Test_Y_stance, ignore_index=True)

# encoding my labels from text to numbers not necessary for stance
encoder = LabelEncoder()
encoder.fit(tags)
tagst=encoder.fit_transform(tags)

num_classes = int((len(set(tagst))))
print((len(set(tagst))))

# tranforming my labels to their encoded numbers
Train_Y_stance = encoder.fit_transform(Train_Y_stance)
Test_Y_stance = encoder.fit_transform(Test_Y_stance)

3


C:\Users\hazem\AppData\Local\Temp\ipykernel_13452\3164755007.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tags = Train_Y_stance.append(Test_Y_stance, ignore_index=True)


In [5]:
Train_Y_stance= keras.utils.to_categorical(Train_Y_stance,num_classes)
Test_Y_stance = keras.utils.to_categorical(Test_Y_stance, num_classes)


num_labels = int(len(Train_Y_stance.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size

In [6]:
import keras.backend as K
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

from keras.metrics import Precision , Recall , Accuracy , TruePositives , TrueNegatives , FalsePositives , FalseNegatives

In [7]:
# Build the model
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])

In [9]:
batch_size = 100
epochs = 2

history = model.fit(Train_X, Train_Y_stance,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


model.save('my_model.h1')

Epoch 1/2
63/63 [==============================] - 17s 249ms/step - loss: 0.5965 - categorical_accuracy: 0.7946 - recall: 0.8814 - precision: 0.6578 - f1_metric: 0.7548 - true_positives: 5543.0000 - true_negatives: 9694.0000 - false_positives: 2884.0000 - false_negatives: 746.0000 - val_loss: 0.5233 - val_categorical_accuracy: 0.8054 - val_recall: 0.9342 - val_precision: 0.6178 - val_f1_metric: 0.7439 - val_true_positives: 653.0000 - val_true_negatives: 994.0000 - val_false_positives: 404.0000 - val_false_negatives: 46.0000
Epoch 2/2
63/63 [==============================] - 15s 245ms/step - loss: 0.1394 - categorical_accuracy: 0.9596 - recall: 0.9924 - precision: 0.7771 - f1_metric: 0.8727 - true_positives: 6241.0000 - true_negatives: 10788.0000 - false_positives: 1790.0000 - false_negatives: 48.0000 - val_loss: 0.6251 - val_categorical_accuracy: 0.8097 - val_recall: 0.9099 - val_precision: 0.6773 - val_f1_metric: 0.7771 - val_true_positives: 636.0000 - val_true_negatives: 1095.0000 - 

INFO:tensorflow:Assets written to: my_model.h1\assets


INFO:tensorflow:Assets written to: my_model.h1\assets


In [10]:
#model = keras.models.load_model('my_model.h1')
Evaluation_valus = model.evaluate(Test_X,Test_Y_stance,verbose=0)
print("Loss" , 'categorical_accuracy','Recall','Precision','f1_metric','TruePositives','TrueNegatives','FalsePositives','FalseNegatives')

print(Evaluation_valus)

Loss categorical_accuracy Recall Precision f1_metric TruePositives TrueNegatives FalsePositives FalseNegatives
[0.6505663990974426, 0.8009999990463257, 0.8930000066757202, 0.660014808177948, 0.7605701088905334, 893.0, 1540.0, 460.0, 107.0]


In [11]:
import csv

Test_X = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['text']
Test_Y_stance = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['stance']

In [12]:
fields = ['Text', 'Predicted', 'Actual','isCorrect']

filename = "../Results/AraBert.csv"

correct_count = 0

with open(filename, 'w',encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)

    for i in range(len(Test_X)):
        tokens = tokenizer.texts_to_matrix([Test_X[i]], mode='tfidf')
        predictions_vector =model.predict(np.array(tokens))
        predicted_class_encoded =np.argmax(predictions_vector,axis=1)
        predicted_class_name = encoder.inverse_transform(predicted_class_encoded)[0]

        isCorrect = "False"
        if predicted_class_name == Test_Y_stance[i]:
            correct_count += 1
            isCorrect = "True"
        
        csvwriter.writerow([Test_X[i], str(predicted_class_name), str(Test_Y_stance[i]), isCorrect])

        # break


1/1 [==============================] - 0s 23ms/step


In [13]:
accuracy = correct_count / len(Test_X)
print("Accuracy: ", accuracy)

Accuracy:  0.801
